# Single QPU VQE experiment

This notebook goes through the running of a VQE algorithm using interlin-q. For simplicity, we only try out using one QPU.

## Step 1: Import libraries.

First we import all the necessary libraries. Interlin-q is build using the python framework [QuNetSim](https://arxiv.org/abs/2003.06397) which is a python software framework that can be used to simulate quantum networks up to the network layer. We also need PennyLane's chemistry library for decomposing the Hamiltonian.

In [13]:
import sys
import numpy as np
sys.path.append("../../")

from qunetsim.components import Network
from qunetsim.objects import Logger

from interlinq import (ControllerHost, Constants, Clock,
Circuit, Layer, ComputingHost, Operation)

from hamiltonian_decomposition import decompose

Logger.DISABLED = False

## Step 2: Decompose the Hamiltonian.

In [2]:
geometry = 'h2.xyz'
charge = 0
multiplicity = 1
basis_set = 'sto-3g'
name = 'h2'

In [3]:
observables, qubit_num = decompose(name, geometry, charge, multiplicity, basis_set)

observables

[[('Identity', 0)],
 [('PauliZ', 0)],
 [('PauliZ', 1)],
 [('PauliZ', 2)],
 [('PauliZ', 3)],
 [('PauliZ', 0), ('PauliZ', 1)],
 [('PauliY', 0), ('PauliX', 1), ('PauliX', 2), ('PauliY', 3)],
 [('PauliY', 0), ('PauliY', 1), ('PauliX', 2), ('PauliX', 3)],
 [('PauliX', 0), ('PauliX', 1), ('PauliY', 2), ('PauliY', 3)],
 [('PauliX', 0), ('PauliY', 1), ('PauliY', 2), ('PauliX', 3)],
 [('PauliZ', 0), ('PauliZ', 2)],
 [('PauliZ', 0), ('PauliZ', 3)],
 [('PauliZ', 1), ('PauliZ', 2)],
 [('PauliZ', 1), ('PauliZ', 3)],
 [('PauliZ', 2), ('PauliZ', 3)]]

## Step 3: Running Circuit on Interlin-q

### Single-step approach

In [4]:
def rotational_gate(params):
    phi, theta, omega = params
    cos = np.cos(theta / 2)
    sin = np.sin(theta / 2)
    return np.array([[np.exp(-1j * (phi + omega) / 2) * cos, -np.exp(1j * (phi - omega) / 2) * sin], 
                     [np.exp(-1j * (phi - omega) / 2) * sin, np.exp(1j * (phi + omega) / 2) * cos]])

def initialise_and_create_ansatz(q_map, parameters):
    layers = []
    host_id = list(q_map.keys())[0]
    
    # Initialise the qubits on the computing host
    ops = []
    
    op = Operation(
        name=Constants.PREPARE_QUBITS,
        qids=q_map[host_id],
        computing_host_ids=[host_id])
    ops.append(op)
    
    # Prepare the qubits on the computing host
    op = Operation(
        name=Constants.SINGLE,
        qids=[q_map[host_id][0]],
        gate=Operation.X,
        computing_host_ids=[host_id])
    ops.append(op)
    
    op = Operation(
        name=Constants.SINGLE,
        qids=[q_map[host_id][1]],
        gate=Operation.X,
        computing_host_ids=[host_id])
    ops.append(op)
    
    layers.append(Layer(ops))
    
    ########################################################
    
    # Apply the ansatz
    for i in range(len(q_map[host_id])):
        ops = list()
        
        op = Operation(
            name=Constants.SINGLE,
            qids=[q_map[host_id][i]],
            gate=Operation.CUSTOM,
            gate_param=rotational_gate(parameters[i]),
            computing_host_ids=[host_id])
        
        ops.append(op)
    
    layers.append(Layer(ops))
    
    op = Operation(
        name=Constants.TWO_QUBIT,
        qids=[q_map[host_id][2], q_map[host_id][3]],
        gate=Operation.CNOT,
        computing_host_ids=[host_id])
    
    layers.append(Layer([op]))
    
    op = Operation(
        name=Constants.TWO_QUBIT,
        qids=[q_map[host_id][2], q_map[host_id][0]],
        gate=Operation.CNOT,
        computing_host_ids=[host_id])
    
    layers.append(Layer([op]))
    
    op = Operation(
        name=Constants.TWO_QUBIT,
        qids=[q_map[host_id][3], q_map[host_id][1]],
        gate=Operation.CNOT,
        computing_host_ids=[host_id])
    
    layers.append(Layer([op]))
    
    ########################################################
    
    ops = []
    # Measuring only the first qubit
    op = Operation(
        name=Constants.MEASURE,
        qids=['q_0_0'],
        cids=['q_0_0'],
        computing_host_ids=[host_id])
    ops.append(op)
    layers.append(Layer(ops))
    
    # Measuring all qubits
    #q_ids = q_map[host_id].copy()
    #ops = []
    #for q_id in q_ids:
    #    op = Operation(
    #        name=Constants.MEASURE,
    #        qids=[q_id],
    #        cids=[q_id],
    #        computing_host_ids=[computing_host_ids[0]])
    #    ops.append(op)
    #layers.append(Layer(ops))
    
    circuit = Circuit(q_map, layers)
    return circuit

In [5]:
def controller_host_protocol(host, q_map, params):
    """
    Protocol for the controller host
    """
    
    monolithic_circuit = initialise_and_create_ansatz(q_map, params)

    host.generate_and_send_schedules(monolithic_circuit)
    
    host.receive_results()

    results = host.results

    print(results)
    print(list(results['QPU_0']['bits'].values()))

In [6]:
def computing_host_protocol(host):
    host.receive_schedule()
    host.send_results()

In [7]:
network = Network.get_instance()
network.delay = 0
network.start()

clock = Clock()

controller_host = ControllerHost(
    host_id="host_1",
    clock=clock
)

computing_hosts, q_map = controller_host.create_distributed_network(
    num_computing_hosts=1,
    num_qubits_per_host=4)
controller_host.start()

network.add_hosts([
    computing_hosts[0],
    controller_host])

2021-03-24 03:11:15,559: Host QPU_0 started processing
2021-03-24 03:11:15,560: Host host_1 started processing


In [8]:
params = np.random.rand(4, 3)

In [11]:
t1 = controller_host.run_protocol(
    controller_host_protocol,
    (q_map, params))
t2 = computing_hosts[0].run_protocol(computing_host_protocol)

t1.join()
t2.join()

2021-03-24 03:11:45,503: host_1 sends BROADCAST message
2021-03-24 03:11:45,659: sending ACK:2 from QPU_0 to host_1
2021-03-24 03:11:45,697: QPU_0 received {"QPU_0": [{"name": "PREPARE_QUBITS", "qids": ["q_0_0", "q_0_1", "q_0_2", "q_0_3"], "cids": null, "gate": null, "gate_param": null, "computing_host_ids": ["QPU_0"], "pre_allocated_qubits": false, "layer_end": 7}, {"name": "SINGLE", "qids": ["q_0_0"], "cids": null, "gate": "X", "gate_param": null, "computing_host_ids": ["QPU_0"], "pre_allocated_qubits": false, "layer_end": 7}, {"name": "SINGLE", "qids": ["q_0_1"], "cids": null, "gate": "X", "gate_param": null, "computing_host_ids": ["QPU_0"], "pre_allocated_qubits": false, "layer_end": 7}, {"name": "SINGLE", "qids": ["q_0_3"], "cids": null, "gate": "custom_gate", "gate_param": [[[0.9418392348578628, -0.3275094061608439], [-0.0736024530229415, 0.016097312431972922]], [[0.0736024530229415, 0.016097312431972922], [0.9418392348578628, 0.3275094061608439]]], "computing_host_ids": ["QPU_0"

7
8
9
10
11
12
13
{'QPU_0': {'type': 'result', 'bits': {'q_0_0': 1}}}
[1]
2
dict_keys([7, 8, 9, 10, 11, 12])


In [12]:
# This should be 7 after the first invocation. 
# For any further invocations, it would increase by 6: 13,19, 25, etc. So is that expected behaviour?
clock.ticks

14

### Split-steps approach

In [ ]:
def prepare_qubits(q_map):
    layers = []
    host_id = list(q_map.keys())[0]
    
    # Initialise the qubits on the computing host
    ops = []
    
    op = Operation(
        name=Constants.PREPARE_QUBITS,
        qids=q_map[host_id],
        computing_host_ids=[host_id])
    ops.append(op)
    
    # Prepare the qubits on the computing host
    op = Operation(
        name=Constants.SINGLE,
        qids=[q_map[host_id][0]],
        gate=Operation.X,
        computing_host_ids=[host_id])
    ops.append(op)
    
    op = Operation(
        name=Constants.SINGLE,
        qids=[q_map[host_id][1]],
        gate=Operation.X,
        computing_host_ids=[host_id])
    ops.append(op)
    
    layers.append(Layer(ops))
    
    circuit = Circuit(q_map, layers)
    return circuit

In [ ]:
def rotational_gate(params):
    phi, theta, omega = params
    cos = np.cos(theta / 2)
    sin = np.sin(theta / 2)
    return np.array([[np.exp(-1j * (phi + omega) / 2) * cos, -np.exp(1j * (phi - omega) / 2) * sin], 
                     [np.exp(-1j * (phi - omega) / 2) * sin, np.exp(1j * (phi + omega) / 2) * cos]])


def apply_ansatz(q_map, parameters):
    layers = []
    host_id = list(q_map.keys())[0]
    
    # Initialise the qubits on the computing host
    ops = []
    
    # Apply ansatz
    for i in range(len(q_map[host_id])):
        ops = list()
        
        op = Operation(
            name=Constants.SINGLE,
            qids=[q_map[host_id][i]],
            gate=Operation.CUSTOM,
            gate_param=rotational_gate(parameters[i]),
            computing_host_ids=[host_id])
        
        ops.append(op)
    
    layers.append(Layer(ops))
    
    op = Operation(
        name=Constants.TWO_QUBIT,
        qids=[q_map[host_id][2], q_map[host_id][3]],
        gate=Operation.CNOT,
        computing_host_ids=[host_id])
    
    layers.append(Layer([op]))
    
    op = Operation(
        name=Constants.TWO_QUBIT,
        qids=[q_map[host_id][2], q_map[host_id][0]],
        gate=Operation.CNOT,
        computing_host_ids=[host_id])
    
    layers.append(Layer([op]))
    
    op = Operation(
        name=Constants.TWO_QUBIT,
        qids=[q_map[host_id][3], q_map[host_id][1]],
        gate=Operation.CNOT,
        computing_host_ids=[host_id])
    
    layers.append(Layer([op]))
    
    circuit = Circuit(q_map, layers)
    return circuit

In [ ]:
def measure(q_map):
    layers = []
    host_id = list(q_map.keys())[0]
    
    ops = []
    
    #q_ids = q_map[host_id].copy()
    #for q_id in q_ids:
    #    print(q_id)
    #print(host_id)
    
    op = Operation(
        name=Constants.MEASURE,
        qids=['q_0_0'],
        cids=['q_0_0'],
        computing_host_ids=[host_id])

    layers.append(Layer([op]))
    
    circuit = Circuit(q_map, layers)
    return circuit

In [ ]:
def controller_host_protocol_preparation(host, q_map, params):
    """
    Protocol for the controller host
    """
    host.generate_and_send_schedules(prepare_qubits(q_map))
    
    #host.receive_results()

In [ ]:
def controller_host_protocol_ansatz(host, q_map, params):
    """
    Protocol for the controller host
    """
    host.generate_and_send_schedules(apply_ansatz(q_map, params))
    
    #host.receive_results()

In [ ]:
def controller_host_protocol_measure(host, q_map, params):
    host.generate_and_send_schedules(measure(q_map))

    host.receive_results()

    print(host.results)

### Computing and Network Set-up

In [ ]:
# Used for the first two invocations
def computing_host_protocol_rec(host):
    host.receive_schedule()

In [ ]:
# Used for the last one
def computing_host_protocol(host):
    host.receive_schedule()
    host.send_results()

In [ ]:
network = Network.get_instance()
network.delay = 0
network.start()

clock = Clock()

controller_host = ControllerHost(
    host_id="host_1",
    clock=clock
)

computing_hosts, q_map = controller_host.create_distributed_network(
    num_computing_hosts=1,
    num_qubits_per_host=4)
controller_host.start()

network.add_hosts([
    computing_hosts[0],
    controller_host])

In [ ]:
params = np.random.rand(4, 3)

In [ ]:
t1 = controller_host.run_protocol(
    controller_host_protocol_preparation,
    (q_map, params))
t2 = computing_hosts[0].run_protocol(computing_host_protocol_rec)

t1.join()
t2.join()

In [ ]:
clock.ticks

In [ ]:
t1 = controller_host.run_protocol(
    controller_host_protocol_ansatz,
    (q_map, params))
t2 = computing_hosts[0].run_protocol(computing_host_protocol_rec)

t1.join()
t2.join()

In [ ]:
clock.ticks

In [ ]:
# For some reason, this returns empty results, not sure why. 
# And even a print statement in the _process_measurement function is not reached
t1 = controller_host.run_protocol(
    controller_host_protocol_measure,
    (q_map, params))
t2 = computing_hosts[0].run_protocol(computing_host_protocol)

t1.join()
t2.join()

In [ ]:
clock.ticks

### Misc

In [ ]:
for _ in range(20):
    network = Network.get_instance()
    network.delay = 0
    network.start()

    clock = Clock()

    controller_host = ControllerHost(
        host_id="host_1",
        clock=clock,
    )

    computing_hosts, q_map = controller_host.create_distributed_network(
        num_computing_hosts=1,
        num_qubits_per_host=4)
    controller_host.start()

    network.add_hosts([
        computing_hosts[0],
        controller_host])
    
    t1 = controller_host.run_protocol(
        controller_host_protocol,
        (q_map, params))
    t2 = computing_hosts[0].run_protocol(computing_host_protocol)

    t1.join()
    t2.join()
    network.stop(False)